# Scratch Paper

This is where I hold the test code that I run inside the Visualize notebook

In [6]:
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug

/root/Documents


<IPython.core.display.Javascript object>

## Most Useful

In [ ]:
import inspect
lines = inspect.getsource(checkit.add_class)
print(lines)

In [ ]:
JS("""
//cy.removeListener('tap', handler);
//cy.removeListener( "click" ) 
//cy.removeListener( "tap" )
//cy.removeListener( "select" )
//cy.removeListener( "unselect" )
""")

## Other

checking observe

In [ ]:
x = []
checkit = ipw.FloatText(description="label", continuous_update=True)
checkit.myVar="howdy"
def checkitFunc(change):
        global x
        x = change
        print(change['new'], type(change['new']))
checkit.observe(checkitFunc, names='value')

checkit
x['new']
checkit.get_state

In [ ]:

JS(
cyto.layout({ name: 'circle'}).run();
)


# Load Stuff

In [83]:
""" DEBUG """
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug
%run "Source Code/Setup Source.ipynb"   #<-- Debug

/root/Documents


# Helper:  <font color="blue">asyncSleep</font> 
The helper function <font color="blue">asyncSleep</font> allows for a cell to wait while style allowing other python and javascripts to run in the background. The primary use of this is to give enought time for <font color="orange">javascript code</font> to run in the background, before preceding to the next cell.

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def mycoro():
    await asyncio.sleep(.1)
    return 

def asyncSleep():
    aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

# <font color="hotpink">Variables</font>  <font color="green">Widgets</font>  (fancy)

These are the variables widgets that I used before I added the *"Retreive Variables"* section. While this setup look fancier, it requires the user to do more steps to use. It also requires a lot more code. This si why I removed it.

### <font color="hotpink">Show Variables</font> 

In [ ]:
Graph.w.varBox = ipw.VBox([])
Graph.w.varBox

In [ ]:
#print(Graph.w.vars)
Graph.w.varBox.vars = [] #<-- where we store list of varBox's labels

We Create a way to add variables to the HBox

In [ ]:
def add(label, value):
    Graph.w.varBox.vars.append( label ) #<-- labels follow same order as children of varBox
    
    # We create the row (HBox) that is added to the box of variables
    var_label = ipw.HTML("<font size=3 style='justify-content: center;'>"+label+" : </font>") 
    var_value = ipw.Text(value,continuous_update=True) if type(value)== type('') else ipw.FloatText(value,continuous_update=True)
    var_btn   = ipw.Button( button_style='danger', description="Remove", layout= ipw.Layout(width='auto') )
    var = ipw.HBox([ var_label, var_value, var_btn ])
    
    # now we setup remove button
    var_btn.label = label
    def remove(b):
        # Remove variable object from  varBox
        child_idx = Graph.w.varBox.vars.index(b.label)  #<- index of child
        Graph.w.varBox.vars.pop( child_idx )    #<-- remove label from varBox.vars
        childs = list(Graph.w.varBox.children)
        childs.pop( child_idx )
        Graph.w.varBox.children = tuple(childs)
        # remove data object from selected elments
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.removeData('"""+b.label+"""')
        }
        """))  
    var_btn.on_click( remove )
    
    # add a Variable Updating Function
    var_value.label = label
    def update(change):
        val =  "'"+change['new']+"'" if type(change['new'])== type('') else str(change['new'])
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.data('"""+change['owner'].label+"""', """+val+""" )
        }
        """))
    var_value.observe( update, names='value')        
        
    # Add the row to the varBox
    Graph.w.varBox.children = Graph.w.varBox.children + (var,)
    
Graph.w.varBox.add = add  

### <font color="hotpink">Add Variable</font> <font color="red">Add Show</font>

In [ ]:
Graph.w.var_add_start = ipw.ToggleButton( button_style='success', icon="plus", layout= ipw.Layout(width='auto') )
Graph.w.var_add_name = ipw.Dropdown( options=Graph.w.vars, layout= ipw.Layout(width='10%') )
Graph.w.var_add_text = ipw.Text()
Graph.w.var_add_num  = ipw.FloatText()
Graph.w.var_add_btn = ipw.Button( button_style='primary', description="Add", layout= ipw.Layout(width='auto') )
Graph.w.var_add_error = ipw.HTML("<h4 class='text-danger'>Empty Text</h4>", layout= ipw.Layout(visibility='hidden') )

Graph.w.var_add = ipw.HBox([ Graph.w.var_add_start, Graph.w.var_add_name, Graph.w.var_add_text, Graph.w.var_add_num,
                             Graph.w.var_add_btn, Graph.w.var_add_error  ])
Graph.w.var_add.layout.align_items = 'center'

for w in Graph.w.var_add.children[1:-1] :  #<-- hides all boxes except "start button" and "error box" (error box already hidden)
    w.layout.display = "none" # <--  Other choice is: None
Graph.w.var_add

In [ ]:
def setVarName(b):
    # TO DO: var_add_name should only display variables that are not already set
    
    val = Graph.w.var_add_name.value 
    if Graph.w.var_add_name.value  == 'Number':
        Graph.w.var_add_text.layout.display = "none"
        Graph.w.var_add_num.layout.display = None
    else:
        Graph.w.var_add_text.layout.display = None
        Graph.w.var_add_num.layout.display = "none"
        
    Graph.w.var_add_btn.layout.display = None
    
Graph.w.var_add_name.observe( setVarName )

In [ ]:
def addVarStart(b):
    if Graph.w.var_add_start.value :  #<-- when activated
        Graph.w.var_add_start.icon = "minus"
        Graph.w.var_add_start.button_style = "danger"
        Graph.w.var_add_name.layout.display = None
        Graph.wf.setVarName("dummy")
    else:                             #<-- when deactivated
        Graph.w.var_add_start.icon = "plus"
        Graph.w.var_add_start.button_style = "success"
        for w in reversed(Graph.w.var_add.children[1:-1]) :  #<-- hides elements, used "reversed" because it looks better when hiding
            w.layout.display = "none"
        # Reset Boxes
        Graph.w.var_add_text.value=''; Graph.w.var_add_num.value=0; Graph.w.var_add_error.layout.visibility='hidden'
        
Graph.w.var_add_start.observe( addVarStart )

In [ ]:
def addVar(b):
    
    if Graph.w.var_add_text.value=='' and Graph.w.var_add_name.value=='Text' : #<-- if user didn't write anything, stop function
        Graph.w.var_add_error.layout.visibility = 'visible'
        return  
    val = Graph.w.var_add_text.value if Graph.w.var_add_name.value=='Text' else Graph.w.var_add_num.value
    val_js = "'"+val+"'" if Graph.w.var_add_name.value=='Text' else str(val)

    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        ele.data( '"""+Graph.w.var_add_name.label+"""', """+val_js+""" )
    }
    """))
    
    Graph.w.varBox.add( Graph.w.var_add_name.label, val )

Graph.w.var_add_btn.on_click( addVar )

# Limiting "Observe" calls 

we limit the number of time that "observe" is called per second for jupyter functions

In [ ]:
def size(b):
    if time.time() - Graph.w.time > .01:  #<-- make function run only ever 1/10 a second
        Graph.w.time = time.time()
        values = "['"+str(Graph.w.size.value)+"','"+str(Graph.w.size.value)+"']" #<-- values for changing node's style
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isNode() ){
                cy.io.style.change( ele, ["width","height"], """+values+"""   )   
            }else{
                cy.io.style.change( ele, "width", '"""+str(Graph.w.size.value/6)+"""'   )   
            }
        }
        """))           
Graph.wf.size = size
Graph.w.size.observe( Graph.wf.size )

#  <font color="teal">Style</font> <font color="orange">Functions</font> 
The following code sets up the <font color="orange">javascript</font> functions/variables that handle the style nodes and edges
Create object that holds style stuff. Add to it the default style of nodes and edges

## <font color="orange">Old Set Style</font>

In [ ]:
JS("""
cy.io.style.set = function(ele, self=false){
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;
    
    if( self == false ){ //<-- set properties of element
        for (var property in style ) {
            if( style[property] != "NaNpx" && !property.includes("pie") ){   // removes error messages from weird stuff in cytoscape.js
                ele.style( property, style[property] )
           }
        }    
    }

};
""")

# Helper: <font color="magenta">JS Box</font>

This box was created to solve the a bunch of the glitches with the "running javascript commands" **<font color="red">Explain More</font>**

In [ ]:
jsBox = JupyterCell( "Javascript Box", "js_num" )
def run(text):
    jsBox.text = text
    jsBox.execute()
jsBox.run = run

In [ ]:
Javascript("alert('howdy ho')")

<font color="red">For debugging</font>

In [ ]:
# Javascript Box
JS(jsBox.text) 
clear_output()

# Helper: <font color="magenta">Output</font><font color="orange">.JS</font> <font color="red">unfinished</font>

**<font color="magenta">OutputJS</font>** is a very useful <font color="magenta">python class</font>/<font color="orange">javascript function</font> that allows variable to go from: <font color="orange">javascript</font> $\rightarrow$ <font color="blue">python</font>.

**Python Code**
```python
JS("OutputJS( 'howdy ho!' )")
OutputJS.var
```
** Output**
```
'howdy ho!'
```

In [ ]:
class Output():
    var = ""
    JS("""
    function sleep(ms){ 
        return new Promise( resolve => setTimeout(resolve, ms)); 
    }; 
    """)
    JS("""
    async function OutputJS(text){ 
        Jupyter.notebook.kernel.execute( 'Output.var = "'+String(text)+'"' );  //<-- write text to python variable: Output.var 
        count = 1
        while( Jupyter.notebook.kernel_busy && count < 25  ){                  //<-- wait for variable to be written, then continue
            console.log( Jupyter.notebook.kernel_busy  )
            console.log( count )
            count += 1
            await sleep(100);
        }
        console.log("one more sleep")
        console.log( Jupyter.notebook.kernel_busy )
        await sleep(1000);
    };  
    """)
    
    def JS(text):
        JS("""OutputJS("""+text+""")""")
        time.sleep(.1)
        return Output.var
        

In [ ]:
Output.var

In [ ]:
JS("OutputJS(x)")
time.sleep(10)
print(var)

In [ ]:
Output.var

## TRY 2

In [ ]:
js_sent = False
js_object = {}
count = 0

In [ ]:
def commFunc(comm, msg):
    @comm.on_msg
    def _recv(msg):
        global js_sent; global js_object
        js_object = msg["content"]["data"]
        js_sent = True
        # Use msg['content']['data'] for the data in the message

get_ipython().kernel.comm_manager.register_target('js_comm', commFunc)

JS("comm = Jupyter.notebook.kernel.comm_manager.new_comm('js_comm',{'foo': 6})")

In [ ]:
aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def myFunc():
    global js_sent; global count
    count = 0
    while( (not js_sent)  and count<20 ):
        time.sleep(.1)
        count += 1

async def waiter( js_object ):
    global js_sent
    js_sent = False
    JS("comm.send("+js_object+")")    
    await commFunc
    return
     

def getJS( text ):
    global js_object
    aio_pool.submit( aio_loop.run_until_complete, waiter(text)  ).result()
    return js_object

## Try 3

In [ ]:
def commFunc(comm, msg):
    @comm.on_msg
    def _recv(msg):
        global js_sent; global js_object
        js_object = msg["content"]["data"]
        js_sent = True
        # Use msg['content']['data'] for the data in the message

get_ipython().kernel.comm_manager.register_target('js_comm', commFunc)
JS("comm = Jupyter.notebook.kernel.comm_manager.new_comm('js_comm',{'foo': 6})")

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import IPython

In [ ]:
aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def myFunc():
    global js_sent; global count
    count = 0
    while( (not js_sent)  and count<20 ):
        time.sleep(.1)
        count += 1

async def waiter( js_object ):
    global js_sent
    js_sent = False
    JS("comm.send("+js_object+")")    
    await commFunc
    return
     

def getJS( text ):
    global js_object
    aio_pool.submit( aio_loop.run_until_complete, waiter(text)  ).result()
    return js_object

In [ ]:
JS("comm.send({'foo':1})") 

In [ ]:
x = getJS("{'foo':6}")
x

In [ ]:
count

In [ ]:
js_sent

In [ ]:
comm.

In [ ]:
x= 0
print(x)
JS("Jupyter.notebook.kernel.execute('x=20 ')")
x=1
print(x)
time.sleep(2)
print(x)

## ASYNCIO

[example](https://github.com/ipython/ipython/issues/11030) and a [tutorial](https://stackabuse.com/python-async-await-tutorial/)


In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def mycoro():
    await asyncio.sleep(1)
    return 5

result = aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

## COMMS

[tutorial](http://jupyter-notebook.readthedocs.io/en/latest/comms.html)

## Part 1

In [20]:
JS("Jupyter.notebook.kernel.comm_manager.register_target('my_comm_target', function(comm, msg) {});")

In [21]:
from ipykernel.comm import Comm
global x

# Use comm to send a message from the kernel
my_comm = Comm(target_name='my_comm_target')
my_comm.send({'foo': 2})

# Add a callback for received messages.
@my_comm.on_msg
def _recv(msg):
    x = "ja"
    # Use msg['content']['data'] for the data in the message

In [22]:
my_comm.send({'foo': 2})

In [15]:

lines = inspect.getsource(my_comm.open)
print(lines)

    def open(self, data=None, metadata=None, buffers=None):
        """Open the frontend-side version of this comm"""
        if data is None:
            data = self._open_data
        comm_manager = getattr(self.kernel, 'comm_manager', None)
        if comm_manager is None:
            raise RuntimeError("Comms cannot be opened without a kernel "
                        "and a comm_manager attached to that kernel.")

        comm_manager.register_comm(self)
        try:
            self._publish_msg('comm_open',
                              data=data, metadata=metadata, buffers=buffers,
                              target_name=self.target_name,
                              target_module=self.target_module,
                              )
            self._closed = False
        except:
            comm_manager.unregister_comm(self)
            raise



<bound method Comm.open of <ipykernel.comm.comm.Comm object at 0x7f5ac4633be0>>

In [75]:
checkit = ipw.FloatText()

In [76]:
checkit.continuous_update

False

## Part 2

In [ ]:
my_comm.comm_id

In [ ]:
my_comm.send({'foo': 2})

In [ ]:
x["content"]["data"]

In [ ]:
comm.send({'foo': 5})

In [ ]:
display("hello")

In [ ]:
get_ipython().asdf

## Try: 8/13

Chekcing if I can get async io to work with a jupyter widget

**Idea:** then use "commm send" on the JS side to change widget value, and an "on change" widget func.

Tried to do widget inside of async didn't wokr

In [14]:
global w
w = ipw.IntSlider( continuous_update=True)
def changeX(b):
    x = 
w.observe( changeX, names="value" )
def()           
Graph.w.size.observe( size )

IntSlider(value=0)

In [26]:
g= 0
w = ipw.IntSlider( continuous_update=True )
def update(change):
    global g
    #val =  "'"+change['new']+"'" if type(change['new'])== type('') else str(change['new'])
    g = change['new']
    
    print(g)
w.observe( update, names='value')   
display(w)

IntSlider(value=0)

2
32
41
40
43
48
52
58
59


Global Variable with asyncio

In [ ]:
import asyncio
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from datetime import datetime
import time


def day_limits():
        variable.value = 90
        print ('Day Variable: ',variable.value)

def night_limits():
        variable.value = 65
        print ('Night Variable: ',variable.value)


def thread_2():
    while True:
        c_hour = int(datetime.now().strftime("%H"))
        c_min = int(datetime.now().strftime("%M"))
        c_sec = int(datetime.now().strftime("%S"))

        print ('%02d:%02d:%02d - Variable: %d ' % (c_hour,c_min,c_sec,variable.value))

        time.sleep(2)


if __name__ == "__main__":
    variable = multiprocessing.Value('i', 60)

    scheduler = AsyncIOScheduler()
    scheduler.add_job(day_limits, 'cron', hour=7,misfire_grace_time=3600,timezone='GB')
    scheduler.add_job(night_limits, 'cron', hour=19, minute=32,misfire_grace_time=3600,timezone='GB')
    scheduler.start()

    scheduler.print_jobs()

In [54]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def mycoro():

    import multiprocessing
    variable = multiprocessing.Value('i', 60)
    
    # Setup widget
    w = ipw.IntSlider( continuous_update=True )
    display(w)
    def update(change):
        variable.value = change['new']
    w.observe( update, names='value')  
    
    # setup async wait
    count=0
    for i in range(5) :
        await asyncio.sleep(.5)
        count+= 1
        print( variable.value )
    return

result = aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

IntSlider(value=0)

60
60
60
60
60


Now trying comm inside of widget

In [101]:
%gui asyncio

In [102]:
%gui asyncio
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [127]:
%gui asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

from ipywidgets import IntSlider
slider = IntSlider()

display( slider )
async def f():
    x = await wait_for_change(slider, 'value')
    print("DONE")

#result = aio_pool.submit(aio_loop.run_until_complete, asyncio.ensure_future(f()) ).result()
#asyncio.run_until_complete, f() ).result()
asyncio.ensure_future(f())

print("hello")

IntSlider(value=0)

hello
DONE


In [ ]:
print("hello a")

In [128]:
from functools import wraps
def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.

    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f

In [132]:
from ipywidgets import IntSlider, VBox, HTML
slider2=IntSlider()

@yield_for_change(slider2, 'value')
def f():
    for i in range(10):
        print('did work %s'%i)
        x = yield
        print('generator function continued with value %s'%x)
display(slider2)
f()



IntSlider(value=0)

did work 0
generator function continued with value 1
did work 1
generator function continued with value 100
did work 2
generator function continued with value 94
did work 3
generator function continued with value 93
did work 4
generator function continued with value 40
did work 5
generator function continued with value 41
did work 6
generator function continued with value 49
did work 7
generator function continued with value 61
did work 8
generator function continued with value 85
did work 9
generator function continued with value 95


In [69]:
def getJS(comm, msg):
    @comm.on_msg
    def _recv(msg):
        #msg['content']['data']
        w.value = False
        # Use msg['content']['data'] for the data in the message
get_ipython().kernel.comm_manager.register_target('getJS', getJS)

In [70]:
JS("""
comm = Jupyter.notebook.kernel.comm_manager.new_comm('getJS')
""")


In [97]:
w = ipw.ToggleButton( )
display(w)
def update(change):
    if w.value :   #<-- first thing to do
        print("howdy")
        display(Javascript("comm.send( {asdf:2} )"))
        
        for i in range(5) :
            time.sleep(.5)
            print( w.value )
        
    else :
        print("BACK")
w.observe( update, names='value')
w.value = True



ToggleButton(value=False)

howdy


<IPython.core.display.Javascript object>

True
True
True
True
True


In [85]:
w.value

False

In [95]:
w.comm.handle_msg
w.comm.class_get_help

<Logger IPKernelApp (WARNING)>

In [100]:
lines = inspect.getsource(w.comm.)
print(lines)

    @contextmanager
    def hold_sync(self):
        """Hold syncing any state until the outermost context manager exits"""
        if self._holding_sync is True:
            yield
        else:
            try:
                self._holding_sync = True
                yield
            finally:
                self._holding_sync = False
                self.send_state(self._states_to_send)
                self._states_to_send.clear()



In [78]:
wait asyncio.sleep(1)

SyntaxError: invalid syntax (<ipython-input-78-d30ca64e9fa3>, line 1)

In [82]:
import asyncio
await asyncio.sleep(.5)

SyntaxError: invalid syntax (<ipython-input-82-237f3e07c8b5>, line 2)

In [16]:
w.value

89

In [34]:
x = 0
def changeX():
    global x
    x = 1

In [35]:
changeX()
x

1

## Scratch Paper

In [72]:
x
def target_func(comm, msg):
    # comm is the kernel Comm instance
    # msg is the comm_open message

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        global x
        x = ['content']['data']
        # Use msg['content']['data'] for the data in the message

    # Send data to the frontend
    w = interactive(f, a=10, b=20)
    comm.send({'foo': 5})

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

In [33]:

checkit = get_ipython().get_ipython().kernel.comm_manager


In [37]:
checkit.notify_change

<bound method HasTraits.notify_change of <ipykernel.comm.manager.CommManager object at 0x7f5ac46e59b0>>

In [43]:
checkit2 = checkit.comms['1fdfd3ee6877412e8c61df98e467c0aa']

In [71]:
checkit2.trait_metadata()

TypeError: trait_metadata() missing 2 required positional arguments: 'traitname' and 'key'

In [ ]:
comm = Jupyter.notebook.kernel.comm_manager.new_comm('my_comm_target', {'foo': 6})
// Send data
comm.send({'foo': 7})

// Register a handler
comm.on_msg(function(msg) {
    console.log(msg.content.data.foo);
});

In [48]:
lines = inspect.getsource(checkit2.kernel.instance)
print(lines)

    @classmethod
    def instance(cls, *args, **kwargs):
        """Returns a global instance of this class.

        This method create a new instance if none have previously been created
        and returns a previously created instance is one already exists.

        The arguments and keyword arguments passed to this method are passed
        on to the :meth:`__init__` method of the class upon instantiation.

        Examples
        --------

        Create a singleton class using instance, and retrieve it::

            >>> from traitlets.config.configurable import SingletonConfigurable
            >>> class Foo(SingletonConfigurable): pass
            >>> foo = Foo.instance()
            >>> foo == Foo.instance()
            True

        Create a subclass that is retrived using the base class instance::

            >>> class Bar(SingletonConfigurable): pass
            >>> class Bam(Bar): pass
            >>> bam = Bam.instance()
            >>> bam == Bar.instance()
          

In [26]:
def target_func(comm, msg):
    # comm is the kernel Comm instance
    # msg is the comm_open message

    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        # Use msg['content']['data'] for the data in the message

    # Send data to the frontend
    comm.send({'foo': 5})

get_ipython().kernel.comm_manager.register_target('my_comm_target', target_func)

IndentationError: expected an indented block (<ipython-input-26-2552cd3c2387>, line 11)

# IPAD Keyboard

Using Juptyer

In [ ]:
JS("""
Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', 'jupyter-notebook:change-cell-to-raw');
""")


In [ ]:
JS("""
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('r');
""")

In [ ]:
"ctrl-H"

Using Raw Javascrip

In [1]:
JS("""
var checkit = function(event) {
    console.log( event.which )
}
$(document).on("keydown", );
""")

<IPython.core.display.Javascript object>

In [ ]:
JS("""
$(document).off("keydown", checkit );
""")

In [ ]:
$(document).on("keydown", function(event) {
  if (event.which == 37) {  // Left Arrow Has Been Pressed

  } else if (event.which == 38) {               // Up Arrow Has Been Pressed
     if( Jupyter.notebook.mode == "command"){
         Jupyter.notebook.select_prev()
     }

  } else if (event.which == 39) {  // Right Arrow Has Been Pressed


  } else if (event.which == 40) {             // Down Arrow Has Been Pressed
     if( Jupyter.notebook.mode == "command"){
         Jupyter.notebook.select_next()
     }
  }
  // prevent the default action
  //event.preventDefault();
});

In [ ]:
    $(document).on("keydown", function(event) {
      if (event.which == 37) {  // Left Arrow Has Been Pressed
        
      } else if (event.which == 38) {               // Up Arrow Has Been Pressed
         if( Jupyter.notebook.mode == "command"){
             Jupyter.notebook.select_prev()
         }
        
      } else if (event.which == 39) {  // Right Arrow Has Been Pressed
                                     
                                     
      } else if (event.which == 40) {             // Down Arrow Has Been Pressed
         if( Jupyter.notebook.mode == "command"){
             Jupyter.notebook.select_next()
         }
      }
      // prevent the default action
      //event.preventDefault();
    });

In [ ]:
%%js
if( /Android|webOS|iPhone|iPad|iPod|BlackBerry|IEMobile|Opera Mini/i.test(navigator.userAgent) ) {
    $(document).on("keydown", function(event) {
      if (event.which == 37) {  // Left Arrow Has Been Pressed
        
      } else if (event.which == 38) {               // Up Arrow Has Been Pressed
         if( Jupyter.notebook.mode == "command"){
             Jupyter.notebook.select_prev()
         }
        
      } else if (event.which == 39) {  // Right Arrow Has Been Pressed
                                     
                                     
      } else if (event.which == 40) {             // Down Arrow Has Been Pressed
         if( Jupyter.notebook.mode == "command"){
             Jupyter.notebook.select_next()
         }
      }
      // prevent the default action
      //event.preventDefault();
    });
}


I wanted to use this to launch key commands from ipad

Simple mapping of single key presses

In [ ]:
$(document).on("keydown", function(event) {
  if (event.which == 37) {
    // Left Arrow Has Been Pressed
  } else if (event.which == 38) {
    // Up Arrow Has Been Pressed
  } else if (event.which == 39) {
    // Right Arrow Has Been Pressed
  } else if (event.which == 40) {
    // Down Arrow Has Been Pressed
  }
  // prevent the default action
  event.preventDefault();
});

Launching commands from pressing 2 keys, (like Control-E for escaple)

In [ ]:
var map = {68: false, 69: false, 86: false};
$(document).keydown(function(e) {
    if (e.keyCode in map) {
        map[e.keyCode] = true;
        if (map[68] && map[69] && map[86]) {
            // FIRE EVENT
        }
    }
}).keyup(function(e) {
    if (e.keyCode in map) {
        map[e.keyCode] = false;
    }
});

# Old Widgets

## <font color="green">Control Panel</font> 

**<font color="green">widget : </font>**

In [ ]:
Graph.w.ControlPanel= widgets.ToggleButtons(
    options=[ 'Edit','Search','Graph Style'])
#Graph.w.ControlPanel

**<font color="blue">Function : </font>**  <font color="blue">ControlChanged</font> changes the editorBox display when the Control Panel changes.

In [ ]:
def ControlChanged(b):
    #widg = Graph.w.ControlPanel.value.replace(" ", "")
    #Graph.editor_box.widg = getattr(Graph.w, widg).show
    #Graph.editor_box.execute()
    #clear_output()
    #display(Graph.w.ControlPanel)
    pass

    
Graph.w.ControlPanel.observe( ControlChanged )

How to change button style

```python
Graph.w.ControlPanel.value = "Search"
```

## <font color="green">All nodes</font>

In [ ]:
Graph.w.Edit.allNodes = ipw.Button(description='All Nodes')

In [ ]:
def allNodes(b):
    JS("cy.nodes().select()")   #<-- delete elements
Graph.wf.allNodes = allNodes
Graph.w.Edit.allNodes.on_click( Graph.wf.allNodes )

## <font color="green">All edges</font>

In [ ]:
Graph.w.Edit.allEdges = ipw.Button(description='All Edges')

In [ ]:
def allEdges(b):
    JS("cy.edges().select()")   #<-- delete elements
Graph.wf.allEdges = allEdges
Graph.w.Edit.allEdges.on_click( Graph.wf.allEdges )

## <font color="green">Add Node</font>

In [ ]:
Graph.w.AddNode = type("widget",(),{})()

In [ ]:
Graph.w.AddNode.head = ipw.HTML("""<h4>Describe the New Node, then Click <font color="blue">Add</font></h4>""")
Graph.w.AddNode.id = ipw.Text(description='ID :')
Graph.w.AddNode.button = ipw.Button(description='Add', button_style='info')
Graph.w.AddNode.show = ipw.VBox( [Graph.w.AddNode.head, Graph.w.AddNode.id, Graph.w.AddNode.button ] )
Graph.w.AddNode.show

### <font color="blue">addNode</font>

<font color="blue">addNode</font> adds a nodes to the network

**<font color="red">Function NOT Implemented</font>**

In [ ]:
def AddNode(b):
    w_id= Graph.w.AddNode.id.value
    JS("""
    cy.add([
      { group: "nodes", data: { id: '{w_id}' } , position: { x: 100, y: 200 } },
    ]);
    """)
Graph.wf.AddNode = AddNode
Graph.w.AddNode.button.on_click( Graph.wf.AddNode )  #<-- function when widget is clicked

## <font color="green">Add Edge</font>

In [ ]:
Graph.w.AddEdge = type("widget",(),{})()
Graph.w.AddEdge.show = ipw.HTML("""<h4 class="text-danger">Widget NOT Implemented</h4>""")

### <font color="blue">addEdge</font>

<font color="blue">addEdge</font> adds an edge to the network

**<font color="red">Function NOT Implemented</font>**

In [ ]:
def AddEdge(b):#self, source, target, id):
    """JS(
    cy.add([
      { group: "edges", data: { id: "e0", source: "n0", target: "n1" } }
    ]);
    )"""
Graph.wf.AddEdge = AddEdge

# Previous Default Style

This style was what was previously called inside of the **Graph.start** command

In [ ]:
    style: [ { selector: 'node', style: { 'height': 20, 'width': 20, 'background-color': 'gray' } },
     { selector: 'edge', style: { 'width': 5, 'opacity': 0.5, 'line-color': 'gray'} } ]

#  JS commands

## <font color="orange">selection Highligh</font>


This functions makes the nodes change color when they are selected

In [ ]:
def selectColor(self):
    JS("""
    cy.on("select", 'node' , function(){
        this.style({'background-blacken': .4 });
    });
    cy.on("unselect", 'node' , function(){
        this.style({'background-blacken': 0 });
    });
    """)
JS_Helper.selectColor = selectColor

## Editor_Box changer (tried to stop flashes)

In [ ]:
JS("""
    var check = 0;
    var check_var;
    cy.io.last_type = "Nothing";

    cy.io.test = function(event){
        cy.io.selection = cy.$(':selected')                 //<-- Get selected elements
        cy.io.last_type = cy.io.select_type 
         
        if( cy.io.selection.length == 0  ){                 //<-- Do nothing if nothing's selected
            return
        } else if( cy.io.selection.length == 1 ){           //<-- If selection lenght is 1
        
            if( cy.io.selection.isNode() && cy.io.last_type != "1 node" ){                      //<-- If it's a node
                console.log("selection changed")
                cy.io.select_type = "1 node"
                Jupyter.notebook.kernel.execute("Graph.editor_box.widg = Graph.w.Edit.boxAll")
                IPython.notebook.execute_cells([cy.io.edit_num])  //<-- cy.io.edit_num was setup by the Graph.editor_box
                
            } else if(  cy.io.last_type != "1 edge" ){                                              //<-- If it's an edge
                cy.io.select_type = "1 edge"
                Jupyter.notebook.kernel.execute("Graph.editor_box.widg = Graph.w.Edit.boxAll")
                IPython.notebook.execute_cells([cy.io.edit_num])  //<-- cy.io.edit_num was setup by the Graph.editor_box
            }
            
        } else {                                           //<-- If Many elements
            console.log("many items")
        }
        
    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")

## When Testing Stuff:

TEST: Add nodes

In [ ]:
JS("""
    cy.add([
      { group: "nodes" },
    ]);
    
    cy.layout({ name: 'circle'}).run();
""")

TEST: Get clicked node info to console

In [ ]:
JS("""

cy.nodes().on("click", function(){
  console.log( this )
});
""")

TEST: get more specific node info:

In [ ]:
JS("""

cy.nodes().on("click", function(){
  console.log( this.data() );
  console.log( " " );console.log( " " );
  console.log( this.style );
});
""")

TEST: Change color on click

In [ ]:
JS("""

cy.nodes().on("click", function(){
    this.style('background-color', 'magenta');
});
""")

TEST: Another color change

In [ ]:
JS("""

cy.nodes().on("click", function(){
    this.style({'background-color': 'blue'});
});
""")

TEST: Get node position

In [ ]:
JS("""

cy.nodes().on("click", function(){
    console.log( this.position() )
});
""")

In [ ]:
JS("""

cy.nodes().on("click", function(){
    console.log( this.style() )
});
""")



TEST: Remove Listeners

In [ ]:
JS("""
cy.nodes().removeListener( "click" )
cy.nodes().removeListener( "select" )
cy.removeListener( "tap" )
""")

TEST: Selected Element

In [ ]:
JS("""
console.log( cy.$(':selected') )
""")

TEST: Jquery commands [Things you can search for](http://js.cytoscape.org/#notation/elements-json) ... [Mayber things u search for?](http://js.cytoscape.org/#selectors/state)

In [ ]:
JS("""
console.log( cy.$(':selectable') )
""")

**TEST: Change Something on select [Things you can select](http://js.cytoscape.org/#events/user-input-device-events)**

In [ ]:
JS("""

cy.nodes().removeListener( "select" )

cy.nodes().on("select", function(){
    this.style({'background-blacken': .3 });
});

cy.nodes().on("unselect", function(){
    this.style({'background-blacken': 0 });
});
""")

** TEST: Add nodes when clicking **

In [ ]:
JS("""
    var checkit;
    
    
    cy.removeListener( "tap" )            // <-- Clears all previous cy "tap" evens
    cy.on('tap', function(event){        // <-- on "tapping" canvas
      if( event.target === cy ){         // <-- If we tapped on the background

          checkit = cy.add([ {  group: "nodes", position: event.position  } ]);            

      } 
    });
""")

**TEST: Add Edge on double click**

In [ ]:
JS("""
cy.removeListener( "tap" )  
""")

In [ ]:
JS("""
    cy.io.click_hist = ["1", "2", "3", "4"];
    
    cy.removeListener( "click" )                               // <-- Clears all previous cy "tap" evens
    cy.on("click" , function(event){                              // <-- on "tapping" canvas
    
      cy.io.click_hist.shift();                                         //<-- Remove oldest click
      
      if( event.target !== cy && event.target.isNode() ){         // <-- If we tapped a Node
          cy.io.click_hist.push( event.target.id() );                  // <-- Add node id to click history
          
          if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] ){      //<-- If 2 double clicks
              
              cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]); //<-- add edge
          }
          
          
      }else{                                           //<-- if we didn't click a node
          cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
      
      } 
      
    });
""")